# Installation

In [ ]:
! pip install flordb

# Getting Started

We start by selecting (or creating) a `git` repository to save our model training code as we iterate and experiment. Flor automatically commits your changes on every run, so no change is lost. Below we provide a sample repository you can use to follow along:

In [ ]:
import os
!git clone git@github.com:ucbepic/ml_tutorial ../ml_tutorial
os.chdir('../ml_tutorial/')

Run the `train.py` script to train a small linear model, 
and test your `flordb` installation.

In [ ]:
! python train.py

Flor will manage checkpoints, logs, command-line arguments, code changes, and other experiment metadata on each run (More details [below](#storage--data-layout)). All of this data is then expesed to the user via SQL or Pandas queries.


# View your experiment history
From the same directory you ran the examples above, open an iPython terminal, then load and pivot the log records.


In [ ]:
import flor
flor.pivot()

# Run some more experiments
The `train.py` script has been prepared in advance to define and manage four different hyper-parameters:

In [ ]:
%cat train.py | grep flor.arg | pygmentize -l python

You can control any of the hyper-parameters (e.g. `hidden`) using Flor's command-line interface:

In [ ]:
! python train.py --kwargs hidden=250

# Application Programming Interface (API)

FlorDB is shipped with utilities for serializing and checkpointing PyTorch state,
and utilities for resuming, auto-parallelizing, and memoizing executions from checkpoint.

The model developer passes objects for checkpointing to `flor.checkpointing(**kwargs)`,
and gives it control over loop iterators by 
calling `flor.loop(name, iterator)` as follows:

In [ ]:
!cat train.py | grep -A 17 flor.checkpointing | pygmentize -l python

As shown, 
we wrap both the nested training loop and main loop with `flor.loop` so Flor can manage their state. Flor will use loop iteration boundaries to store selected checkpoints adaptively, and on replay time use those same checkpoints to resume training from the appropriate epoch.  


### Logging API

You call `flor.log(name, value)` and `flor.arg(name, default=None)` to log metrics and register tune-able hyper-parameters, respectively. 

In [ ]:
%cat train.py | grep -C 1 -e 'flor.arg' -e 'flor.log' | pygmentize -l python

The `name`(s) you use for the variables you intercept with `flor.log` and `flor.arg` will become a column (measure) in the full pivoted view (see [Viewing your exp history](#view-your-experiment-history)).
